<a href="https://colab.research.google.com/github/FalpClinicaTI/Bigdata_UBO/blob/main/BigData_Trabajo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")
gdrive='/content/gdrive/MyDrive/BigData/Clase1'

Mounted at /content/gdrive


In [ ]:
!pip install pyspark

In [ ]:
!pip install pyarrow

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DecimalType
from pyspark.sql.types import StringType
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField

segmentacionShema = StructType ([
    StructField("RUT_CLI", IntegerType(), True),
    StructField("SEGMENTO", StringType(), True),
    StructField("SUB_SEGMENTO_1", DecimalType(), True),
    StructField("SUB_SEGMENTO_2", DecimalType(), True),
    StructField("SUB_SEGMENTO_3", DecimalType(), True),
    StructField("VALOR_JERARQUIA", StringType(), True),
    StructField("JERARQUIA", StringType(), True),
    StructField("SEGMENTO_string", StringType(), True)
])

pricingShema = StructType ([
    StructField("SEGMENTO", IntegerType(), True),
    StructField("SUCURSAL", StringType(), True),
    StructField("RUBRO", StringType(), True),
    StructField("FAMILIA", IntegerType(), True),
    StructField("SUBFAMILIA", IntegerType(), True),
    StructField("PRODUCTO", StringType(), True),
    StructField("JERARQUIA", StringType(), True),
    StructField("MARGEN", DecimalType(), True),
    StructField("MARGEN_PRE_APRO", DecimalType(), True)
])



In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").appName("ejemplo1").getOrCreate()

In [ ]:
segmentacion = gdrive+'/Tabla_segmentacion.csv'
pricing = gdrive+'/Tabla_pricing.csv'

In [ ]:
#df_segmentacion=spark.read.format("com.databricks.spark.csv").schema(customsShema).load(segmentacion)
# Cargar el DataFrame con otro nombre para la columna
df_segmentacion = spark.read.format("com.databricks.spark.csv").schema(segmentacionShema).load(segmentacion)
df_pricing = spark.read.format("com.databricks.spark.csv").schema(pricingShema).load(pricing)

In [ ]:
# Nos envia sin estructura
#df_segmentacion = spark.read.csv(segmentacion, inferSchema=True, header =True,sep=";")
df_segmentacion = spark.read.csv(segmentacion, segmentacionShema, header =True,sep=";")
#df_pricing = spark.read.csv(pricing, inferSchema=True, header =True,sep=";")
df_pricing = spark.read.csv(pricing, pricingShema, header =True,sep=";")

#df_pricing

In [ ]:
df_segmentacion.createOrReplaceTempView("segmentacion")
df_pricing.createOrReplaceTempView("pricing")

In [ ]:
spark.sql("select * from segmentacion").show()

+--------+--------+--------------+--------------+--------------+---------------+---------+---------------+
| RUT_CLI|SEGMENTO|SUB_SEGMENTO_1|SUB_SEGMENTO_2|SUB_SEGMENTO_3|VALOR_JERARQUIA|JERARQUIA|SEGMENTO_string|
+--------+--------+--------------+--------------+--------------+---------------+---------+---------------+
|25501902|   31833|             0|             0|             3|           5-10|        2|     0-0-3-p90+|
|77447639|   31844|             0|             0|             5|           5-10|        2|      0-0-5-p90|
|10935354|   51716|             3|             0|             3|           5-15|        2|       3-0-p90+|
|76956256|   31820|             0|             0|             1|           5-19|        2|      0-0-1-p90|
|76131545|   31825|             0|             0|             2|           5-21|        2|      0-0-2-p70|
|77310489|   31832|             0|             0|             3|           5-10|        2|      0-0-3-p90|
|76660860|   31844|             0|   

In [ ]:
spark.sql("select count(*) from segmentacion").show()

+--------+
|count(1)|
+--------+
| 2805667|
+--------+



In [ ]:
spark.sql("select * from pricing").show()

+--------+--------+-----+-------+----------+--------+---------+------+---------------+
|SEGMENTO|SUCURSAL|RUBRO|FAMILIA|SUBFAMILIA|PRODUCTO|JERARQUIA|MARGEN|MARGEN_PRE_APRO|
+--------+--------+-----+-------+----------+--------+---------+------+---------------+
|   31820|    E523|  I4 |     13|         2|    NULL|        2|     0|              0|
|   31820|    E523|  I5 |     13|         2|    NULL|        2|     0|              0|
|   31820|    E523|  O4 |     13|         2|    NULL|        2|     0|              0|
|   31821|    E523|  C7 |     13|         2|    NULL|        2|     0|              0|
|   31822|    E523|  I3 |     13|         2|    NULL|        2|     0|              0|
|   31823|    E523|  C6 |     13|         2|    NULL|        2|     0|              0|
|   31823|    E523|  I2 |     13|         2|    NULL|        2|     0|              0|
|   31823|    E523|  O4 |     13|         2|    NULL|        2|     0|              0|
|   31824|    E523|  C1 |     13|         2

In [ ]:
spark.sql("select count(*) from pricing").show()

+--------+
|count(1)|
+--------+
|12581326|
+--------+



In [ ]:
qry='''
  select * from pricing
  where SEGMENTO= (
    select SEGMENTO
    from segmentacion
    where RUT_CLI=25501902
    limit 1)
  and FAMILIA=13 and SUBFAMILIA=2 and SUCURSAL='E566'
    '''
spark.sql(qry).show()

+--------+--------+-----+-------+----------+--------+---------+------------------+------------------+
|SEGMENTO|SUCURSAL|RUBRO|FAMILIA|SUBFAMILIA|PRODUCTO|JERARQUIA|            MARGEN|   MARGEN_PRE_APRO|
+--------+--------+-----+-------+----------+--------+---------+------------------+------------------+
|   31833|    E566|  C3 |     13|         2|    NULL|        2|0.3842230218422212|0.3350387557238239|
|   31833|    E566|  C4 |     13|         2|    NULL|        2|0.3765971203726944|0.3464784282842897|
|   31833|    E566|  I1 |     13|         2|    NULL|        2|0.3831501831501831|0.3394390383514596|
|   31833|    E566|  I3 |     13|         2|    NULL|        2|0.3347916186967534|0.2968197879858657|
|   31833|    E566|  O2 |     13|         2|    NULL|        2|0.3835990068026814|0.3435897435897436|
|   31833|    E566|  C1 |     13|         2|    NULL|        2|0.3720893094560769|0.3392901158491496|
|   31833|    E566|  C6 |     13|         2|    NULL|        2|0.3666363126926311|